# Lattice PDF

In this notebook, we will use the architecture developed previously to variationally find ground hadron states, $|P\rangle$ and use these to generate the parton distribution function, $f_{q/h}(x)$

The lattice PDF is given as: $$f_{q/h}(x) = \int dz e^{ixP^+z} \langle P| \zeta(z) |P\rangle$$ with $$\zeta(z) = \sum_{i,j = 0}^1 e^{iHt}\chi^\dagger_{2(z + i)}e^{-iHt}\chi_{2j + 1}  $$

We utilize lightcone coordinates, so to keep this correlation funciton on the lightcone, we set $t = z$. $z$ represents the physical continuum coordinate, i.e. $z\in\mathbb{R}$ Mapping this to the lattice, we have to double the number of lattice points for each physical coordinate (to utilize the staggered fermion approach of ADD CITATION), i.e. $N = 2Z$ (capital letters correspond to cutoffs, $0 \leq z \leq Z$). For each $N$, we have to use $2N$ qubits to distinguish the two colors of red and green. 

In [1]:
from symmer import PauliwordOp, QuantumState
from symmer.evolution import trotter
from qiskit.quantum_info import SparsePauliOp
from hamiltonian import H
from basis_states import *
from openfermion import *

In [5]:
N = 4

In [6]:
full_ham = H(N)

In [7]:
mesons, baryons = get_basis(N)

In [8]:
oh_bin_dict = one_hot_to_binary_encoding(mesons)

In [9]:
Ham_meson = H_sector(full_ham, mesons)

/Users/carterm.gustin/anaconda3/envs/phd/lib/python3.9/site-packages/symmer/operators/base.py:400: ComplexWarning: Casting complex values to real discards the imaginary part
  temp_mat[:matrix.shape[0],


In [10]:
vals, vecs = np.linalg.eigh(Ham_meson.to_sparse_matrix.toarray())

In [11]:
nonzero_vals = []
non_trivial_vecs = []

for i in range(len(vals)):
    if vals[i] != 0:
        nonzero_vals.append(vals[i])
        non_trivial_vecs.append(vecs[:, i])

In [12]:
psi_vacuum = QuantumState.from_array(vecs[:, 0].reshape([-1, 1]))
psi_meson = QuantumState.from_array(non_trivial_vecs[1].reshape([-1, 1]))
meson_energy = nonzero_vals[1]
vacuum_energy = nonzero_vals[0]

In [13]:
P_0 = revert_to_full_basis(psi_vacuum, oh_bin_dict, full_ham.n_qubits)
P_meson = revert_to_full_basis(psi_meson, oh_bin_dict, full_ham.n_qubits)

In [14]:
def zeta(z_val, num_qubits):
    n = 2*z_val
    zeta_op = PauliwordOp.empty(num_qubits)

    exp_op_plus = full_ham.multiply_by_constant(1j * z_val)
    exp_op_minus = full_ham.multiply_by_constant(-1j * z_val)

    for i in range(0, 2):
        for j in range(0, 2):
            qubit_string_create = str(2 * (n + i)) + "^"
            qubit_string_annihilate = str(2 * j + 1)
            pauli_create = PauliwordOp.from_openfermion(jordan_wigner(FermionOperator(qubit_string_create)))
            pauli_annihilate = PauliwordOp.from_openfermion(jordan_wigner(FermionOperator(qubit_string_annihilate)))
            
            if pauli_create.n_qubits < num_qubits:
                missing_qubits_create = num_qubits - pauli_create.n_qubits
                pauli_create = \
                    pauli_create.tensor(PauliwordOp.from_dictionary({'I'*missing_qubits_create: 1.0}))
            if pauli_annihilate.n_qubits < num_qubits:
                missing_qubits_annihilate = num_qubits - pauli_annihilate.n_qubits
                pauli_annihilate = \
                    pauli_annihilate.tensor(PauliwordOp.from_dictionary({'I'*missing_qubits_annihilate: 1.0}))
            zeta_op += trotter(exp_op_plus) * pauli_create * trotter(exp_op_minus) * pauli_annihilate
    return zeta_op

In [15]:
Z = zeta(1, full_ham.n_qubits)

In [15]:
P_meson.dagger * Z * P_meson

0j